# 머신러닝 with Python 
## 09 앙상블 학습 부스팅-LightGBM 연습- breast_cancer 데이터

##### 이번에는 LightGBM를 활용해 유방암 여부를 예측하는 모형을 만들어본다.

(책 p.286~ 에서 그래디언트 부스팅을 실습하였었다. -08번 파일)

---


## LightGBM

LightGBM은 마이크로소프트에서 개발한 분산 그래디언트 부스팅 프레임 워크로 무료 오픈 소스.

GBM: Gradient Boosting Model 의 약자이다.

Boosting하는 방식에 크게 2가지가 있다. 
```
    1. AdaBoost와 같이 중요한 데이터(일반적으로 모델이 틀린 데이터)에 대해 weight를 주는 방식

    2. GBDT와 같이 loss function처럼 정답지와 오답지간의 차이를 반복적으로 training하는 방식 

       (즉, gradient를 이용해서 모델을 개선하는 방식 (XGBoost와 LightGBM이 해당))
```
*출처: https://mac-user-guide.tistory.com/79 [🌷나의 선인장🌵]*

그러나 XGBoost와 LightGBM에도 큰 차이가 있는데, 

XGBoost는 **균형 트리 분할 방식(level wise)**을 사용한다. 최대한 균형잡힌 트리를 만들기 떄문에 트리의 깊이를 최소화 할 수 있다. 다만 시간이 더 걸린다. 

LightGBM은 **리프 중심 트리 분할 방식(leaf Wise)**을 사용한다. 균형 보다는 최대 손실 값(max delta loss)을 가지는 리프 노드를 지속적으로 분할하면서 

트리가 깊어지고 비대칭적인 트리를 만들게 된다. 내가 직관적으로 생각했을 때 그래디언트 부스팅 방식에 떠오늘 방식은 이 방식에 더 가깝기도 했다.

이러한 방식으로 트리를 계속해서 분할하게 되면 결국 균형 트리 분할 방식보다 예측 오류 손실을 최소화 한다는 방법을 이용한 것이라고 한다.

*출처: https://kimdingko-world.tistory.com/184*


 

---

하단부에서는

> 배운점1: 

> 배운점2: 

### 데이터 불러오기

In [1]:
from sklearn import datasets

In [2]:
# 유방암 데이터 가져오기

raw_breast_cancer = datasets.load_breast_cancer()

In [3]:
# 데이터 살펴보기

raw_breast_cancer

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0

In [4]:
# 데이터 셋 내 피처 이름들

raw_breast_cancer.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

### 피처, 타깃 데이터 지정

In [5]:
X = raw_breast_cancer.data
y = raw_breast_cancer.target

### 트레이닝/테스트 데이터 분할

In [6]:
from sklearn.model_selection import train_test_split                 # 분할을 위해 필요한 함수
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 0)    # 분리. randomstate는 고정

### 데이터 표준화

In [7]:
from sklearn.preprocessing import StandardScaler         # 데이터 표준화를 위한 함수
std_scale = StandardScaler()                             # 표준화 스케일러 지정
std_scale.fit(X_tn)                                      # 트레이닝 피처를 기준으로 표준화를 적합

X_tn_std = std_scale.transform(X_tn) 
X_te_std = std_scale.transform(X_te)                     # 트레인, 테스트 데이터 각각 적합시킨 표준화에 맞게 변형

### 데이터 학습

In [8]:
from lightgbm import LGBMClassifier                                  # LightGBM 함수

# 모델 선언
clf_lgbm = LGBMClassifier()

# 모델 훈련
clf_lgbm.fit(X_tn_std, y_tn) 

LGBMClassifier()

In [ ]:
# 조기 중단 기능에 필요한 파라미터 정의
evals = [(X_test, y_test)]
lgbm_wrapper.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='logloss', eval_set=evals, verbose=True)

preds = lgbm_wrapper.predict(X_test)
pred_proba = lgbm_wrapper.predict_proba(X_test)[:,1]

### 데이터 예측

In [9]:
pred_lgbm = clf_lgbm.predict(X_te_std)
print(pred_lgbm)

[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 1 1 1 1 0 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0
 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 1 0 1 0 1 1 0 1 1 1 1 1 1 1 0 1 0 1 0 0 1
 0 0 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 0]


### 정확도 평가

In [10]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_lgbm)                # 실제값과 예측값을 넣음
print(accuracy)

0.9790209790209791


### confusion matrix 확인

In [11]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_lgbm)
print(conf_matrix)

[[51  2]
 [ 1 89]]


### 분류 리포트 확인

In [12]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_lgbm)
print(class_report)

              precision    recall  f1-score   support

           0       0.98      0.96      0.97        53
           1       0.98      0.99      0.98        90

    accuracy                           0.98       143
   macro avg       0.98      0.98      0.98       143
weighted avg       0.98      0.98      0.98       143



### 결과

아무런 옵션도 주지 않은 채 살펴본 결과가 나쁘지 않다. (random_state까지 주지 않았다 왜그랬어;)

그러나 한 가지 마음에 걸리는건 LightGBM은 학습하는데 시간이 적게 걸리고, 메모리 사용량이 상대적으로 적으며, 범주형 변수들을 자동으로 변환하고 최적 분할을 시도하며 GPU 학습도 지원하지만

**dataset이 작을 사용할 경우 과적합 가능성이 높다**는 단점을 가지고 있다는 것이었다. 지금 있는 데이터 셋이 작디 작은 데이터 셋이라서 다른 데이터 셋으로 실험하는게 더 나을 것 같다는 판단.

하지만 -멈출 수 없지- 같은 실험은 해보아야 겠다.

# 실험1: max_depth = 1~5

LightGBM의 default max_depth는 -1이다. 리프 위주로 뻗어나가는 것은 알겠는데, -1인 이유는...뭔지 모르겠어서 만져봐야 겠다.

In [14]:
from lightgbm import LGBMClassifier                              # XGBoost 함수
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


for i in range(-1,7):

    # 모델 선언
    clf_lgbm = LGBMClassifier(max_depth = i, random_state = 0) 

    # 모델 훈련
    clf_lgbm.fit(X_tn_std, y_tn) 

    # 데이터 예측
    pred_lgbm = clf_lgbm.predict(X_te_std)

    # 정확도 평가
    accuracy = accuracy_score(y_te, pred_lgbm)

    # 분류 리포트 확인
    class_report = classification_report(y_te, pred_lgbm)

    # 모델 상태 확인 
    print("분류기 개별 max_depth: ", i )
    print("모델 정확도: ", accuracy)
    print("모델 분류 리포트")
    print(class_report)
    print("========================================================")

분류기 개별 max_depth:  -1
모델 정확도:  0.9790209790209791
모델 분류 리포트
              precision    recall  f1-score   support

           0       0.98      0.96      0.97        53
           1       0.98      0.99      0.98        90

    accuracy                           0.98       143
   macro avg       0.98      0.98      0.98       143
weighted avg       0.98      0.98      0.98       143

분류기 개별 max_depth:  0
모델 정확도:  0.9790209790209791
모델 분류 리포트
              precision    recall  f1-score   support

           0       0.98      0.96      0.97        53
           1       0.98      0.99      0.98        90

    accuracy                           0.98       143
   macro avg       0.98      0.98      0.98       143
weighted avg       0.98      0.98      0.98       143

분류기 개별 max_depth:  1
모델 정확도:  0.9790209790209791
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        53
           1       0.97      1.00      0.98        90

    

 ### 결과
 
 이상한 결과를 맞게 되었는데, max_depth = 3 과 5일 때의 성능이 높게 나온 것은 왠지, 단순한 우연일 것 같다는 생각이 든다.
 
 전체적인 경향은 일정하거나, 성능이 오히려 줄어드는 듯한 경향으로 보였는데 조금씩 높게 나오는 것을
 
 이 적은 데이터를 가지고 좋은 성능이 나오는 최적의 뎁스라고 이야기 할 수 없을 것 같고. 
 
 주어진 데이터에 맞게 억지로 성능을 높이는 일은 대회나가는 일이 아니면 없어야 할 일일 테니까 말이다.
 
 ---
 
 앞서 했었던 부스팅 모델들과 비슷한 실험을 진행해보려고 한다.

# 실험2: n_estimators = 10~200 

default 값은 100이었다.

In [15]:
from lightgbm import LGBMClassifier                              # XGBoost 함수
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

n_esti = [10, 25, 50, 75, 100, 125, 150, 175, 200]

for i in n_esti:

    # 모델 선언
    clf_lgbm = LGBMClassifier(n_estimators = i, random_state = 0) 

    # 모델 훈련
    clf_lgbm.fit(X_tn_std, y_tn) 

    # 데이터 예측
    pred_lgbm = clf_lgbm.predict(X_te_std)

    # 정확도 평가
    accuracy = accuracy_score(y_te, pred_lgbm)

    # 분류 리포트 확인
    class_report = classification_report(y_te, pred_lgbm)

    # 모델 상태 확인 
    print("분류기 개별 max_depth: ", -1 )
    print("분류기 개수: ", i)
    print("모델 정확도: ", accuracy)
    print("모델 분류 리포트")
    print(class_report)
    print("========================================================")

분류기 개별 max_depth:  -1
분류기 개수:  10
모델 정확도:  0.958041958041958
모델 분류 리포트
              precision    recall  f1-score   support

           0       0.96      0.92      0.94        53
           1       0.96      0.98      0.97        90

    accuracy                           0.96       143
   macro avg       0.96      0.95      0.95       143
weighted avg       0.96      0.96      0.96       143

분류기 개별 max_depth:  -1
분류기 개수:  25
모델 정확도:  0.965034965034965
모델 분류 리포트
              precision    recall  f1-score   support

           0       0.94      0.96      0.95        53
           1       0.98      0.97      0.97        90

    accuracy                           0.97       143
   macro avg       0.96      0.96      0.96       143
weighted avg       0.97      0.97      0.97       143

분류기 개별 max_depth:  -1
분류기 개수:  50
모델 정확도:  0.986013986013986
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      0.96      0.98        53
           1       0.98 

### 결과

분류기의 개수가 늘어날 수록 전체적으로 성능이 증가하는 경향을 보이고 있다.